In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# use a pandas data frame to load the csv file
csv_file_path = '/content/drive/MyDrive/traffic_data_preprocessed.csv'

In [5]:
df = pd.read_csv(csv_file_path)
df.head()

,day_of_week,time,time_of_day,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume,day,month,year
0,5,9.0,2,7,-0.404153,1.055720,-0.670405,1.297362,-1.552441,-1.552441,0.612030,-0.008384,-0.032546,-0.269063,1,24,1.157384,1,9,0
1,5,10.0,2,7,0.276572,-0.249763,-0.183967,1.307378,-1.552441,-1.552441,0.692536,-0.008384,-0.032546,0.631348,1,2,0.640677,1,9,0
2,5,11.0,2,7,-0.499693,-0.309103,-0.183967,1.297362,-1.163334,-1.163334,0.708936,-0.008384,-0.032546,1.017238,1,19,0.766716,1,9,0
3,5,12.0,0,7,-1.610349,-0.309103,-0.183967,1.297362,0.003989,0.003989,0.749934,-0.008384,-0.032546,1.017238,1,19,0.896771,1,9,0
4,5,13.0,0,7,1.506654,-0.368443,-0.183967,1.297362,0.782204,0.782204,0.825223,-0.008384,-0.032546,0.631348,1,2,0.842539,1,9,0


In [10]:
# print stats of df
df.describe()

,day_of_week,time,time_of_day,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume,day,month,year
count,33750.000000,33750.000000,33750.000000,33750.000000,3.375000e+04,3.375000e+04,3.375000e+04,3.375000e+04,3.375000e+04,3.375000e+04,3.375000e+04,3.375000e+04,3.375000e+04,3.375000e+04,33750.000000,33750.000000,3.375000e+04,33750.000000,33750.000000,33750.000000
mean,3.003793,11.404385,1.771407,6.997956,-1.541088e-16,-3.368499e-16,8.421247e-18,-9.263372e-18,-1.338978e-16,-1.338978e-16,2.475847e-15,7.442277e-17,4.315889e-17,-4.252730e-17,2.520000,15.959170,1.120026e-16,14.646874,5.407674,2.639822
std,2.004658,6.951078,1.121304,0.143489,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,2.739005,9.081681,1.000015e+00,8.700568,3.543241,1.572845
min,0.000000,0.000000,0.000000,0.000000,-1.729775e+00,-3.454131e+00,-1.643280e+00,-1.997923e+00,-1.552441e+00,-1.552441e+00,-2.087726e+01,-8.383621e-03,-3.254579e-02,-1.298103e+00,0.000000,0.000000,-1.627008e+00,0.000000,0.000000,0.000000
25%,1.000000,5.000000,1.000000,7.000000,-8.579691e-01,-6.651441e-01,-6.704049e-01,-6.958346e-01,-7.742261e-01,-7.742261e-01,-6.224042e-01,-8.383621e-03,-3.254579e-02,-1.272377e+00,0.000000,7.000000,-1.041884e+00,7.000000,2.000000,1.000000
50%,3.000000,11.000000,2.000000,7.000000,1.893822e-03,4.693761e-02,-1.839671e-01,5.289966e-03,3.989074e-03,3.989074e-03,5.994200e-03,-8.383621e-03,-3.254579e-02,3.483615e-01,1.000000,17.000000,4.764441e-02,15.000000,5.000000,3.000000
75%,5.000000,17.000000,3.000000,7.000000,8.736993e-01,8.183595e-01,7.889083e-01,9.067358e-01,7.822043e-01,7.822043e-01,7.864605e-01,-8.383621e-03,-3.254579e-02,1.017238e+00,5.000000,24.000000,8.465567e-01,22.000000,9.000000,4.000000
max,6.000000,23.000000,3.000000,11.000000,1.721620e+00,1.708462e+00,6.139723e+00,1.607860e+00,1.560420e+00,1.560420e+00,2.099910e+00,1.836660e+02,5.222439e+01,1.274498e+00,10.000000,37.000000,2.028605e+00,30.000000,11.000000,5.000000


# Linear Regression

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Assuming df is your DataFrame
# Separate features and target
X = df.drop(columns=["traffic_volume"]).values
y = df["traffic_volume"].values

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)
print("Mean Squared Error (MSE):", mse)
print("R-squared:", r2)

Intercept: 0.8156605850567867
Coefficients: [ 1.93751605e-02  1.49199575e-02 -5.79636532e-01  1.88361239e-02
 -3.42248195e-04 -1.68815126e-03  9.10976331e-03  1.05384244e-02
 -1.31867408e-03 -1.31867408e-03  3.44653583e-02  3.36957698e-03
 -2.66233172e-03  6.31817633e-03 -5.50859941e-03 -2.83089829e-03
 -1.28424985e-03 -9.40596922e-03 -7.48428159e-03]
Mean Squared Error (MSE): 0.495614171712364
R-squared: 0.5078296181379685


# Ridge

In [8]:
from sklearn.linear_model import Ridge, Lasso

ridge_model = Ridge(alpha=10.0)
ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_test)

print("Intercept:", ridge_model.intercept_)
print("Coefficients:", ridge_model.coef_)
print("Mean Squared Error:", mean_squared_error(y_test, ridge_pred))
print("R-squared:", r2_score(y_test, ridge_pred))

Intercept: 0.8175095240413317
Coefficients: [ 1.93729459e-02  1.49315143e-02 -5.79437570e-01  1.85065646e-02
 -3.42475588e-04 -1.68129027e-03  9.10868554e-03  1.05374872e-02
 -1.31835364e-03 -1.31835364e-03  3.44725163e-02  3.36911839e-03
 -2.66099125e-03  6.33210204e-03 -5.51252631e-03 -2.83154919e-03
 -1.28460249e-03 -9.40589480e-03 -7.48265789e-03]
Mean Squared Error: 0.4956183862676608
R-squared: 0.5078254328676348


# Lasso

In [9]:
lasso_model = Lasso(alpha=.001)
lasso_model.fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_test)

print("Intercept:", lasso_model.intercept_)
print("Coefficients:", lasso_model.coef_)
print("Mean Squared Error:", mean_squared_error(y_test, lasso_pred))
print("R-squared:", r2_score(y_test, lasso_pred))

Intercept: 0.9436180593235897
Coefficients: [ 0.01914847  0.01498157 -0.57898859  0.         -0.         -0.00085958
  0.00834104  0.00965461 -0.00167766 -0.          0.03303719  0.00260277
 -0.00153673  0.00461611 -0.00514597 -0.00287276 -0.00127284 -0.00926276
 -0.00690831]
Mean Squared Error: 0.49557007327454955
R-squared: 0.5078734101564728
